In [1]:
#pip install transformers

In [2]:
import numpy as np
import pandas as pd
import torch
import pickle
from open_clip import create_model_from_pretrained, get_tokenizer



In [3]:
model, preprocess = create_model_from_pretrained('hf-hub:timm/ViT-B-16-SigLIP')
tokenizer = get_tokenizer('hf-hub:timm/ViT-B-16-SigLIP')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

def get_text_embeddings(descriptions: list[str]):
    text_tokens = tokenizer(descriptions).to(device)
    with torch.no_grad():
        text_features = model.encode_text(text_tokens).float()
    text_features /= text_features.norm(dim=-1, keepdim=True)
    text_features = text_features.cpu().numpy()
    return text_features

cap3D = pd.read_csv('cap3D_test_set.csv')
cap3D['theme'] = cap3D['theme'].str.replace(' ', '')
cap3D['shape'] = cap3D['shape'].str.replace(' ', '')
cap3D = cap3D[~cap3D.isna().any(axis=1)]
cap3D

,Unnamed: 0,uid,caption,category,color,material,theme,shape
0,0,ed51a51909ee46c780db3a85e821feb2,A green and white rifle.,rifle,Green,Metal,Military,Rectangular
1,1,9110b606f6c547b2980fcb3c8c4b6a1c,"a small building with a roof, accompanied by b...",house,Brown,Wood,Rustic,Rectangular
2,2,80d9caaa1fa04502af666135196456e1,a pair of purple and black swords with white h...,sword,Multicolor,Metal,Oriental,Symmetrical
3,3,28d43a218cd8466a8c1f82b29b71e314,"a small house, island, road with trash, trash ...",island,Multicolored,Mixed,Environmental,Irregular
4,4,75582285fab442a2ba31733f9c8fae66,"a small, grassy hill.",hill,Green,Earth,Nature,Mound
...,...,...,...,...,...,...,...,...
9971,9994,3a0f68d68e0e426780fbe2da8ca06de8,A plaque featuring a star and city name.,plaque,Black,Metal,Urban,Rectangle
9972,9995,aae70af3556d468795220ece999901eb,a wooden bench with a pair of handrails and ra...,bench,Brown,Wood,N/A,Rectangular
9973,9996,e383772896c44539b6a14591be784472,Small blue object with various interpretations...,ambiguous,Blue,N/A,N/A,N/A
9974,9997,bda253a8f4d444bc81c43d0dc9168c8e,a man with purple hair,human,Purple,N/A,N/A,N/A


In [4]:
def create_similarity_label_dict(feature: str, labels: list[str]):
    similarity_dict = {}
    tags = cap3D[feature].unique().tolist()
    label_embeddings = get_text_embeddings(labels)
    tag_embeddings = get_text_embeddings(tags)
    similarities = label_embeddings @ tag_embeddings.T # CLIP Score array of shape n labels x m tags
    for j, tag in enumerate(tags):
        tag_similarities = [similarities[i][j] for i in range(len(labels))] # getting n similarities for jth tag
        similarity_dict[tag] = labels[np.argmax(tag_similarities)]
    return similarity_dict

def create_feature_dict(features):
    feature_dict = {}
    for feature, labels in features.items():
        feature_dict[feature] = create_similarity_label_dict(feature, labels)
    return feature_dict   

shape_theme_material_color = {
    'shape': ['Cuboid', 'Sphere', 'Cone', 'Cylinder'],
    'theme': ['Nature', 'Culture', 'Fantasy', 'Modern'],
    'material': ['Wood', 'Metal', 'Stone', 'Plastic'],
    'color': ['Yellow', 'Red', 'Blue', 'Gray','Multicolor']
}
                                      
stmc = create_feature_dict(shape_theme_material_color)
                               

cap3D2 = cap3D.assign(color_clip = cap3D['color'].apply(lambda x: stmc['color'][x]),
                      material_clip = cap3D['material'].apply(lambda x: stmc['material'][x]),
                      theme_clip = cap3D['theme'].apply(lambda x: stmc['theme'][x]),
                      shape_clip = cap3D['shape'].apply(lambda x: stmc['shape'][x]))

cap3D2

,Unnamed: 0,uid,caption,category,color,material,theme,shape,color_clip,material_clip,theme_clip,shape_clip
0,0,ed51a51909ee46c780db3a85e821feb2,A green and white rifle.,rifle,Green,Metal,Military,Rectangular,Yellow,Metal,Nature,Cuboid
1,1,9110b606f6c547b2980fcb3c8c4b6a1c,"a small building with a roof, accompanied by b...",house,Brown,Wood,Rustic,Rectangular,Red,Wood,Modern,Cuboid
2,2,80d9caaa1fa04502af666135196456e1,a pair of purple and black swords with white h...,sword,Multicolor,Metal,Oriental,Symmetrical,Multicolor,Metal,Modern,Sphere
3,3,28d43a218cd8466a8c1f82b29b71e314,"a small house, island, road with trash, trash ...",island,Multicolored,Mixed,Environmental,Irregular,Multicolor,Stone,Nature,Sphere
4,4,75582285fab442a2ba31733f9c8fae66,"a small, grassy hill.",hill,Green,Earth,Nature,Mound,Yellow,Stone,Nature,Cone
...,...,...,...,...,...,...,...,...,...,...,...,...
9971,9994,3a0f68d68e0e426780fbe2da8ca06de8,A plaque featuring a star and city name.,plaque,Black,Metal,Urban,Rectangle,Gray,Metal,Modern,Cuboid
9972,9995,aae70af3556d468795220ece999901eb,a wooden bench with a pair of handrails and ra...,bench,Brown,Wood,N/A,Rectangular,Red,Wood,Nature,Cuboid
9973,9996,e383772896c44539b6a14591be784472,Small blue object with various interpretations...,ambiguous,Blue,N/A,N/A,N/A,Blue,Stone,Nature,Sphere
9974,9997,bda253a8f4d444bc81c43d0dc9168c8e,a man with purple hair,human,Purple,N/A,N/A,N/A,Red,Stone,Nature,Sphere


In [5]:
def request_gpt4v(message, image):
    payload = prepare_inputs(message, image)
    response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)
#     print(response.json())
    res = response.json()['choices'][0]['message']['content']
    return res

 

In [6]:
with open('embeddings_SigLIP.pkl', 'rb') as f:
    image_embeddings = pickle.load(f)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

CustomTextCLIP(
  (visual): TimmModel(
    (trunk): VisionTransformer(
      (patch_embed): PatchEmbed(
        (proj): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
        (norm): Identity()
      )
      (pos_drop): Dropout(p=0.0, inplace=False)
      (patch_drop): Identity()
      (norm_pre): Identity()
      (blocks): Sequential(
        (0): Block(
          (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
          (attn): Attention(
            (qkv): Linear(in_features=768, out_features=2304, bias=True)
            (q_norm): Identity()
            (k_norm): Identity()
            (attn_drop): Dropout(p=0.0, inplace=False)
            (proj): Linear(in_features=768, out_features=768, bias=True)
            (proj_drop): Dropout(p=0.0, inplace=False)
          )
          (ls1): Identity()
          (drop_path1): Identity()
          (norm2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
          (mlp): Mlp(
            (fc1): Linear(in_features=768

In [7]:
cap3D_uids = cap3D['uid'].tolist()
# Function to generate text embeddings
def get_text_embeddings(descriptions):
    text_tokens = tokenizer(descriptions).to(device)
    with torch.no_grad():
        text_features = model.encode_text(text_tokens).float()
    text_features /= text_features.norm(dim=-1, keepdim=True)
    text_features = text_features.cpu().numpy()
    return text_features

# Function for object retrieval
def retrieve_objects(query):
    # Generate text embedding for the query
    query_embedding = get_text_embeddings([query])
    
    new_embeddings = np.array([image_embeddings[uid]['image_feature'] for uid in cap3D_uids])
    new_embeddings = new_embeddings[:,0,:]
    uid_embeddings = [uid for uid in cap3D_uids]
    
    # Calculate similarity between query embedding and dataset embeddings
    similarities = list(query_embedding @ new_embeddings.T)[0]
    
    grouped_similarities = [(x,y) for x,y in zip(uid_embeddings,similarities)]
    # Rank images based on similarity
    ranked_indices = sorted(grouped_similarities, key=lambda x: x[1],reverse=True)
    
    # Retrieve top results 
    top_results = ranked_indices[:10]  # Adjust number of top results as needed
    return top_results


# Example query
query = "Cat"

# Perform object retrieval
results = retrieve_objects(query)
print(results)

[('d6b0bf73a0d849d08c929f1f7af00805', 0.08209119), ('14cf75cbafcf4a5da3f22067565a821c', 0.080240324), ('6d0eaa4145774c5fb254dcb759d5b29e', 0.074480124), ('2144c95689d84d50b5db84ac361b7d2d', 0.07421863), ('e0a5908bd44e42a4a08afe1ddf43662d', 0.073333845), ('a48202eca385476cafb03a1ed395f2b2', 0.07217845), ('29ecbb4d17c34375b007a3ca4f2f6586', 0.07174268), ('a2c05c8d42194feabc3ad761f15495ec', 0.07003605), ('118f19a1e9a1479285a3141c0d1f9b3f', 0.06881328), ('e8861a671c3245f0a5a9ff54b7c75e91', 0.06761809)]


In [8]:
def get_all_top_tens(uid_list):
    all_rankings = {}
    for uid in uid_list:
        caption = cap3D.loc[cap3D['uid'] == uid, 'caption']
        top_ten = retrieve_objects(caption.iloc[0])
        all_rankings[uid] = top_ten
    return all_rankings    

In [9]:
rankings = get_all_top_tens(cap3D['uid'])

In [10]:
#same function but cap3d2 and color_clip, etc hopefully they improve!
def evaluate_clip(all_objects):
    results = {}
    for uid in all_objects:
        ranking = all_objects[uid]
        top_ten_df = cap3D.loc[cap3D['uid'].isin([x[0] for x in ranking])]
        gt_row = cap3D.loc[cap3D['uid'] == uid]
        compare_color = np.sum(top_ten_df['color'] == gt_row['color'].iloc[0])
        compare_theme = np.sum(top_ten_df['theme'] == gt_row['theme'].iloc[0])
        compare_shape = np.sum(top_ten_df['shape'] == gt_row['shape'].iloc[0])
        compare_material = np.sum(top_ten_df['material'] == gt_row['material'].iloc[0])
        uid_in = uid in top_ten_df['uid'].to_list()
        results[uid] = {'color':compare_color,'theme':compare_theme,
                        'shape':compare_shape,'material':compare_material,'gt_retrieved':uid_in}
        
    return results

In [11]:
clip_evaluations = evaluate_clip(rankings)

In [12]:
cap3D_results = cap3D.assign(color_clip = cap3D['uid'].apply(lambda uid: clip_evaluations[uid]['color']),
                      material_clip = cap3D['uid'].apply(lambda uid: clip_evaluations[uid]['material']),
                      theme_clip = cap3D['uid'].apply(lambda uid: clip_evaluations[uid]['theme']),
                      shape_clip = cap3D['uid'].apply(lambda uid: clip_evaluations[uid]['shape']),
                      gt_retrieved_clip = cap3D['uid'].apply(lambda uid: clip_evaluations[uid]['gt_retrieved']))

In [13]:
cap3D_results

,Unnamed: 0,uid,caption,category,color,material,theme,shape,color_clip,material_clip,theme_clip,shape_clip,gt_retrieved_clip
0,0,ed51a51909ee46c780db3a85e821feb2,A green and white rifle.,rifle,Green,Metal,Military,Rectangular,3,6,8,4,True
1,1,9110b606f6c547b2980fcb3c8c4b6a1c,"a small building with a roof, accompanied by b...",house,Brown,Wood,Rustic,Rectangular,5,7,3,9,False
2,2,80d9caaa1fa04502af666135196456e1,a pair of purple and black swords with white h...,sword,Multicolor,Metal,Oriental,Symmetrical,5,8,1,1,True
3,3,28d43a218cd8466a8c1f82b29b71e314,"a small house, island, road with trash, trash ...",island,Multicolored,Mixed,Environmental,Irregular,4,4,1,7,True
4,4,75582285fab442a2ba31733f9c8fae66,"a small, grassy hill.",hill,Green,Earth,Nature,Mound,9,1,6,1,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9971,9994,3a0f68d68e0e426780fbe2da8ca06de8,A plaque featuring a star and city name.,plaque,Black,Metal,Urban,Rectangle,1,1,2,3,True
9972,9995,aae70af3556d468795220ece999901eb,a wooden bench with a pair of handrails and ra...,bench,Brown,Wood,N/A,Rectangular,6,5,6,8,True
9973,9996,e383772896c44539b6a14591be784472,Small blue object with various interpretations...,ambiguous,Blue,N/A,N/A,N/A,8,7,8,2,False
9974,9997,bda253a8f4d444bc81c43d0dc9168c8e,a man with purple hair,human,Purple,N/A,N/A,N/A,6,9,1,1,True


In [14]:
cap3D_results[cap3D_results.columns[8:]].mean(axis=0)

color_clip           4.616045
material_clip        4.498750
theme_clip           3.211327
shape_clip           3.595934
gt_retrieved_clip    0.708664
dtype: float64

In [15]:
#same function but cap3d2 and color_clip, etc hopefully they improve!
def evaluate_clip_made(all_objects):
    results = {}
    for uid in all_objects:
        ranking = all_objects[uid]
        top_ten_df = cap3D2.loc[cap3D2['uid'].isin([x[0] for x in ranking])]
        gt_row = cap3D2.loc[cap3D2['uid'] == uid]
        compare_color = np.sum(top_ten_df['color_clip'] == gt_row['color_clip'].iloc[0])
        compare_theme = np.sum(top_ten_df['theme_clip'] == gt_row['theme_clip'].iloc[0])
        compare_shape = np.sum(top_ten_df['shape_clip'] == gt_row['shape_clip'].iloc[0])
        compare_material = np.sum(top_ten_df['material_clip'] == gt_row['material_clip'].iloc[0])
        uid_in = uid in top_ten_df['uid'].to_list()
        results[uid] = {'color':compare_color,'theme':compare_theme,
                        'shape':compare_shape,'material':compare_material,'gt_retrieved':uid_in}
        
    return results

In [16]:
cap3D2_results = evaluate_clip_made(rankings)

In [17]:
cap3D2_final = cap3D2.assign(color_clip = cap3D2['uid'].apply(lambda uid: cap3D2_results[uid]['color']),
                      material_clip = cap3D2['uid'].apply(lambda uid: cap3D2_results[uid]['material']),
                      theme_clip = cap3D2['uid'].apply(lambda uid: cap3D2_results[uid]['theme']),
                      shape_clip = cap3D2['uid'].apply(lambda uid: cap3D2_results[uid]['shape']),
                      gt_retrieved_clip = cap3D2['uid'].apply(lambda uid: cap3D2_results[uid]['gt_retrieved']))
cap3D2_final

,Unnamed: 0,uid,caption,category,color,material,theme,shape,color_clip,material_clip,theme_clip,shape_clip,gt_retrieved_clip
0,0,ed51a51909ee46c780db3a85e821feb2,A green and white rifle.,rifle,Green,Metal,Military,Rectangular,3,6,9,4,True
1,1,9110b606f6c547b2980fcb3c8c4b6a1c,"a small building with a roof, accompanied by b...",house,Brown,Wood,Rustic,Rectangular,7,7,8,10,False
2,2,80d9caaa1fa04502af666135196456e1,a pair of purple and black swords with white h...,sword,Multicolor,Metal,Oriental,Symmetrical,5,8,1,10,True
3,3,28d43a218cd8466a8c1f82b29b71e314,"a small house, island, road with trash, trash ...",island,Multicolored,Mixed,Environmental,Irregular,4,6,6,10,True
4,4,75582285fab442a2ba31733f9c8fae66,"a small, grassy hill.",hill,Green,Earth,Nature,Mound,9,5,10,1,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9971,9994,3a0f68d68e0e426780fbe2da8ca06de8,A plaque featuring a star and city name.,plaque,Black,Metal,Urban,Rectangle,4,1,3,8,True
9972,9995,aae70af3556d468795220ece999901eb,a wooden bench with a pair of handrails and ra...,bench,Brown,Wood,N/A,Rectangular,6,7,6,9,True
9973,9996,e383772896c44539b6a14591be784472,Small blue object with various interpretations...,ambiguous,Blue,N/A,N/A,N/A,9,8,8,7,False
9974,9997,bda253a8f4d444bc81c43d0dc9168c8e,a man with purple hair,human,Purple,N/A,N/A,N/A,6,9,3,9,True


In [18]:
#cap3D[(cap3D == 'N/A').apply(lambda x: any(x), axis=1)]
#cap3D2_final[~(cap3D == 'N/A').apply(lambda x: any(x), axis=1),cap3D2_final.columns[8:]].mean(axis=0)

In [19]:
for col in cap3D2_final.columns[8:-1]:
    variable = ~(cap3D2_final[col.replace('_clip','')] == 'N/A')
    print(f'Mean {col}:', cap3D2_final[variable][col].mean())
#     variable = ~(cap3D2_final[col.replace('_clip','')] == 'N/A')
#     print(variable)

Mean color_clip: 5.962278508533537
Mean material_clip: 6.00250027176867
Mean theme_clip: 5.2491416629347665
Mean shape_clip: 6.699186991869919


In [ ]:
#Replace Cap3D2 w this:
# color: ['Basic Solid Colors', 'Metallics', 'Neutrals', 'Multicolor and Patterns', 'Combinations and Dual Tones']
# material: ['Natural Materials', 'Synthetic Materials', 'Metals', 'Minerals & Stones', 'Composite & Mixed Materials']
# theme: ['Cultural-Historical', 'Style-Genre', 'Functional-Practical', 'Natural-Environmental', 'Abstract-Fantastical']
# shape: ['Geometric Shapes', 'Representational Forms', 'Organic Shapes', 'Abstract & Conceptual', 'Functional Objects']

In [20]:
shape_theme_material_color = {
    'shape': ['Geometric Shapes', 'Representational Forms', 'Organic Shapes', 'Abstract & Conceptual', 'Functional Objects'],
    'theme': ['Cultural-Historical', 'Style-Genre', 'Functional-Practical', 'Natural-Environmental', 'Abstract-Fantastical'],
    'material': ['Natural Materials', 'Synthetic Materials', 'Metals', 'Minerals & Stones', 'Composite & Mixed Materials'],
    'color': ['Basic Solid Colors', 'Metallics', 'Neutrals', 'Multicolor and Patterns', 'Combinations and Dual Tones']
}
                                      
stmc = create_feature_dict(shape_theme_material_color)
                               

cap3D3 = cap3D.assign(color_clip = cap3D['color'].apply(lambda x: stmc['color'][x]),
                      material_clip = cap3D['material'].apply(lambda x: stmc['material'][x]),
                      theme_clip = cap3D['theme'].apply(lambda x: stmc['theme'][x]),
                      shape_clip = cap3D['shape'].apply(lambda x: stmc['shape'][x]))

cap3D3

,Unnamed: 0,uid,caption,category,color,material,theme,shape,color_clip,material_clip,theme_clip,shape_clip
0,0,ed51a51909ee46c780db3a85e821feb2,A green and white rifle.,rifle,Green,Metal,Military,Rectangular,Basic Solid Colors,Metals,Natural-Environmental,Geometric Shapes
1,1,9110b606f6c547b2980fcb3c8c4b6a1c,"a small building with a roof, accompanied by b...",house,Brown,Wood,Rustic,Rectangular,Neutrals,Natural Materials,Natural-Environmental,Geometric Shapes
2,2,80d9caaa1fa04502af666135196456e1,a pair of purple and black swords with white h...,sword,Multicolor,Metal,Oriental,Symmetrical,Multicolor and Patterns,Metals,Natural-Environmental,Geometric Shapes
3,3,28d43a218cd8466a8c1f82b29b71e314,"a small house, island, road with trash, trash ...",island,Multicolored,Mixed,Environmental,Irregular,Multicolor and Patterns,Natural Materials,Natural-Environmental,Geometric Shapes
4,4,75582285fab442a2ba31733f9c8fae66,"a small, grassy hill.",hill,Green,Earth,Nature,Mound,Basic Solid Colors,Metals,Natural-Environmental,Geometric Shapes
...,...,...,...,...,...,...,...,...,...,...,...,...
9971,9994,3a0f68d68e0e426780fbe2da8ca06de8,A plaque featuring a star and city name.,plaque,Black,Metal,Urban,Rectangle,Basic Solid Colors,Metals,Natural-Environmental,Geometric Shapes
9972,9995,aae70af3556d468795220ece999901eb,a wooden bench with a pair of handrails and ra...,bench,Brown,Wood,N/A,Rectangular,Neutrals,Natural Materials,Natural-Environmental,Geometric Shapes
9973,9996,e383772896c44539b6a14591be784472,Small blue object with various interpretations...,ambiguous,Blue,N/A,N/A,N/A,Basic Solid Colors,Metals,Natural-Environmental,Functional Objects
9974,9997,bda253a8f4d444bc81c43d0dc9168c8e,a man with purple hair,human,Purple,N/A,N/A,N/A,Basic Solid Colors,Metals,Natural-Environmental,Functional Objects


In [21]:
def evaluate_clip_GPT(all_objects):
    results = {}
    for uid in all_objects:
        ranking = all_objects[uid]
        top_ten_df = cap3D3.loc[cap3D3['uid'].isin([x[0] for x in ranking])]
        gt_row = cap3D3.loc[cap3D3['uid'] == uid]
        compare_color = np.sum(top_ten_df['color_clip'] == gt_row['color_clip'].iloc[0])
        compare_theme = np.sum(top_ten_df['theme_clip'] == gt_row['theme_clip'].iloc[0])
        compare_shape = np.sum(top_ten_df['shape_clip'] == gt_row['shape_clip'].iloc[0])
        compare_material = np.sum(top_ten_df['material_clip'] == gt_row['material_clip'].iloc[0])
        uid_in = uid in top_ten_df['uid'].to_list()
        results[uid] = {'color':compare_color,'theme':compare_theme,
                        'shape':compare_shape,'material':compare_material,'gt_retrieved':uid_in}
        
    return results

In [22]:
cap3D3_results = evaluate_clip_GPT(rankings)

In [23]:
cap3D3_final = cap3D3.assign(color_clip = cap3D3['uid'].apply(lambda uid: cap3D3_results[uid]['color']),
                      material_clip = cap3D3['uid'].apply(lambda uid: cap3D3_results[uid]['material']),
                      theme_clip = cap3D3['uid'].apply(lambda uid: cap3D3_results[uid]['theme']),
                      shape_clip = cap3D3['uid'].apply(lambda uid: cap3D3_results[uid]['shape']),
                      gt_retrieved_clip = cap3D3['uid'].apply(lambda uid: cap3D3_results[uid]['gt_retrieved']))
cap3D2_final

,Unnamed: 0,uid,caption,category,color,material,theme,shape,color_clip,material_clip,theme_clip,shape_clip,gt_retrieved_clip
0,0,ed51a51909ee46c780db3a85e821feb2,A green and white rifle.,rifle,Green,Metal,Military,Rectangular,3,6,9,4,True
1,1,9110b606f6c547b2980fcb3c8c4b6a1c,"a small building with a roof, accompanied by b...",house,Brown,Wood,Rustic,Rectangular,7,7,8,10,False
2,2,80d9caaa1fa04502af666135196456e1,a pair of purple and black swords with white h...,sword,Multicolor,Metal,Oriental,Symmetrical,5,8,1,10,True
3,3,28d43a218cd8466a8c1f82b29b71e314,"a small house, island, road with trash, trash ...",island,Multicolored,Mixed,Environmental,Irregular,4,6,6,10,True
4,4,75582285fab442a2ba31733f9c8fae66,"a small, grassy hill.",hill,Green,Earth,Nature,Mound,9,5,10,1,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9971,9994,3a0f68d68e0e426780fbe2da8ca06de8,A plaque featuring a star and city name.,plaque,Black,Metal,Urban,Rectangle,4,1,3,8,True
9972,9995,aae70af3556d468795220ece999901eb,a wooden bench with a pair of handrails and ra...,bench,Brown,Wood,N/A,Rectangular,6,7,6,9,True
9973,9996,e383772896c44539b6a14591be784472,Small blue object with various interpretations...,ambiguous,Blue,N/A,N/A,N/A,9,8,8,7,False
9974,9997,bda253a8f4d444bc81c43d0dc9168c8e,a man with purple hair,human,Purple,N/A,N/A,N/A,6,9,3,9,True


In [24]:
for col in cap3D3_final.columns[8:-1]:
    variable = ~(cap3D3_final[col.replace('_clip','')] == 'N/A')
    print(f'Mean {col}:', cap3D3_final[variable][col].mean())

Mean color_clip: 6.38406348516143
Mean material_clip: 6.506685509294488
Mean theme_clip: 5.998507239886551
Mean shape_clip: 6.860207768744354
